Using PuLP with pandas and binary constraints to solve a scheduling problem
In this example, we’ll be solving a scheduling problem. We have 2 offshore production plants in 2 locations and an estimated demand for our products.

We want to produce a schedule of production from both plants that meets our demand with the lowest cost.

A factory can be in 2 states:

Off – Producing zero units

On – Producing between its minimum and maximum production capacities.


Both factories have fixed costs, that are incurred as long as the factory is on, and variable costs, a cost per unit of production. These vary month by month.

We also know that factory B is down for maintenance in month 5.

We’ll start by importing our data.

In [1]:
import pandas as pd
import pulp

In [2]:
factories = pd.DataFrame.from_csv('factory_variables.csv', index_col=['Month', 'Factory'])
factories

C:\Users\shakib\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


Max_Capacity  Min_Capacity  Variable_Costs  Fixed_Costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B              120000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

In [3]:
demand = pd.DataFrame.from_csv('monthly_demand.csv', index_col=['Month'])

C:\Users\shakib\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [4]:
production = pulp.LpVariable.dicts("production",
                                     ((month, factory) for month, factory in factories.index),
                                     lowBound=0,
                                     cat='Integer')

In [5]:
production

{(1, 'A'): production_(1,_'A'),
 (1, 'B'): production_(1,_'B'),
 (2, 'A'): production_(2,_'A'),
 (2, 'B'): production_(2,_'B'),
 (3, 'A'): production_(3,_'A'),
 (3, 'B'): production_(3,_'B'),
 (4, 'A'): production_(4,_'A'),
 (4, 'B'): production_(4,_'B'),
 (5, 'A'): production_(5,_'A'),
 (5, 'B'): production_(5,_'B'),
 (6, 'A'): production_(6,_'A'),
 (6, 'B'): production_(6,_'B'),
 (7, 'A'): production_(7,_'A'),
 (7, 'B'): production_(7,_'B'),
 (8, 'A'): production_(8,_'A'),
 (8, 'B'): production_(8,_'B'),
 (9, 'A'): production_(9,_'A'),
 (9, 'B'): production_(9,_'B'),
 (10, 'A'): production_(10,_'A'),
 (10, 'B'): production_(10,_'B'),
 (11, 'A'): production_(11,_'A'),
 (11, 'B'): production_(11,_'B'),
 (12, 'A'): production_(12,_'A'),
 (12, 'B'): production_(12,_'B')}

In [6]:
factory_status = pulp.LpVariable.dicts("factory_status",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')

In [7]:
factory_status

{(1, 'A'): factory_status_(1,_'A'),
 (1, 'B'): factory_status_(1,_'B'),
 (2, 'A'): factory_status_(2,_'A'),
 (2, 'B'): factory_status_(2,_'B'),
 (3, 'A'): factory_status_(3,_'A'),
 (3, 'B'): factory_status_(3,_'B'),
 (4, 'A'): factory_status_(4,_'A'),
 (4, 'B'): factory_status_(4,_'B'),
 (5, 'A'): factory_status_(5,_'A'),
 (5, 'B'): factory_status_(5,_'B'),
 (6, 'A'): factory_status_(6,_'A'),
 (6, 'B'): factory_status_(6,_'B'),
 (7, 'A'): factory_status_(7,_'A'),
 (7, 'B'): factory_status_(7,_'B'),
 (8, 'A'): factory_status_(8,_'A'),
 (8, 'B'): factory_status_(8,_'B'),
 (9, 'A'): factory_status_(9,_'A'),
 (9, 'B'): factory_status_(9,_'B'),
 (10, 'A'): factory_status_(10,_'A'),
 (10, 'B'): factory_status_(10,_'B'),
 (11, 'A'): factory_status_(11,_'A'),
 (11, 'B'): factory_status_(11,_'B'),
 (12, 'A'): factory_status_(12,_'A'),
 (12, 'B'): factory_status_(12,_'B')}

In [8]:
model = pulp.LpProblem("Cost minimising scheduling problem", pulp.LpMinimize)

In [9]:
model += pulp.lpSum(
    [production[month, factory] * factories.loc[(month, factory), 'Variable_Costs'] for month, factory in factories.index]
    + [factory_status[month, factory] * factories.loc[(month, factory), 'Fixed_Costs'] for month, factory in factories.index]
)

In [10]:
# Production in any month must be equal to demand
months = demand.index
for month in months:
    model += production[(month, 'A')] + production[(month, 'B')] == demand.loc[month, 'Demand']

In [11]:
# Production in any month must be between minimum and maximum capacity, or zero.
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

In [12]:
# Factory B is off in May
model += factory_status[5, 'B'] == 0
model += production[5, 'B'] == 0

In [13]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [14]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factory_status[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Factory Status  Production
Month Factory                            
1     A                   1.0     70000.0
      B                   1.0     50000.0
2     A                   1.0     45000.0
      B                   1.0     55000.0
3     A                   1.0     70000.0
      B                   1.0     60000.0
4     A                   1.0     30000.0
      B                   1.0    100000.0
5     A                   1.0    140000.0
      B                   0.0         0.0
6     A                   1.0     60000.0
      B                   1.0     70000.0
7     A                   1.0     90000.0
      B                   1.0     60000.0
8     A                   1.0     70000.0
      B                   1.0    100000.0
9     A                   1.0    100000.0
      B                   1.0    100000.0
10    A                   1.0    190000.0
      B                   0.0         0.0
11    A                   1.0     80000.0
      B                   1.0     60000.0
12    A                   1.0    100000.0
      B                   0.0         0.0

In [15]:
# Print our objective function value (Total Costs)
print(pulp.value(model.objective))

12906400.0
